<a href="https://colab.research.google.com/github/HimanshuSahoo/DATA-SCIENCE/blob/main/Book_Scrap_Project_Base_model_to_read_from_url.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import re

In [ ]:
url = 'https://www.goodreads.com/shelf/show/free-online'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
print(soup)

<!DOCTYPE html>

<html class="desktop withSiteHeaderTopFullImage">
<head>
<title>Free Online Books</title>
<meta content="Books shelved as free-online: The Violet and the Tom by Eve Ocotillo, The Student Prince by FayJay, Heart in Hand by salifiable, Close Protection by Cord..." name="description"/>
<meta content="telephone=no" name="format-detection"/>
<link href="https://www.goodreads.com/shelf/show/free-online" rel="canonical"/>
<script type="text/javascript"> var ue_t0=window.ue_t0||+new Date();
 </script>
<script type="text/javascript">
    var ue_mid = "A1PQBFHBHS6YH1";
    var ue_sn = "www.goodreads.com";
    var ue_furl = "fls-na.amazon.com";
    var ue_sid = "645-3188077-9482682";
    var ue_id = "TDTGJC94KE78PWRB78QK";

    (function(e){var c=e;var a=c.ue||{};a.main_scope="mainscopecsm";a.q=[];a.t0=c.ue_t0||+new Date();a.d=g;function g(h){return +new Date()-(h?0:a.t0)}function d(h){return function(){a.q.push({n:h,a:arguments,t:a.d()})}}function b(m,l,h,j,i){var k={m:m,f:l,l:h

In [ ]:
links = []
title = []
author = []
genre = []
discription = []

In [ ]:
pages = list(range(1,21))

In [ ]:
for page in pages:
    req = requests.get("https://www.goodreads.com/shelf/show/1000-books-to-read-before-you-die?page={}".format(page))
    soup = BeautifulSoup(req.text, 'html.parser')
    book_data = soup.findAll('div', attrs={'class': 'elementList'})
    sleep(randint(2,8))
    for store in book_data:
        name = store.find('a', class_='bookTitle').text
        title.append(name)
        
        writer = store.find('a', class_='authorName').text 
        author.append(writer)
        

In [ ]:
def get_genres(soup):
    genres = []
    for node in soup.find_all('div', {'class': 'left'}):
        current_genres = node.find_all('a', {'class': 'actionLinkLite bookPageGenreLink'})
        current_genre = ' > '.join([g.text for g in current_genres])
        if current_genre.strip():
            genres.append(current_genre)
    return genres


def get_series_name(soup):
    series = soup.find(id="bookSeries").find("a")
    if series:
        series_name = re.search(r'\((.*?)\)', series.text).group(1)
        return series_name
    else:
        return ""


def get_series_uri(soup):
    series = soup.find(id="bookSeries").find("a")
    if series:
        series_uri = series.get("href")
        return series_uri
    else:
        return ""


def get_isbn(soup):
    try:
        isbn = re.findall(r'nisbn: [0-9]{10}' , str(soup))[0].split()[1]
        return isbn
    except:
        return "isbn not found"

def get_isbn13(soup):
    try:
        isbn13 = re.findall(r'nisbn13: [0-9]{13}' , str(soup))[0].split()[1]
        return isbn13
    except:
        return "isbn13 not found"


def get_rating_distribution(soup):
    distribution = re.findall(r'renderRatingGraph\([\s]*\[[0-9,\s]+', str(soup))[0]
    distribution = ' '.join(distribution.split())
    distribution = [int(c.strip()) for c in distribution.split('[')[1].split(',')]
    distribution_dict = {'5 Stars': distribution[0],
                         '4 Stars': distribution[1],
                         '3 Stars': distribution[2],
                         '2 Stars': distribution[3],
                         '1 Star':  distribution[4]}
    return distribution_dict


def get_num_pages(soup):
    if soup.find('span', {'itemprop': 'numberOfPages'}):
        num_pages = soup.find('span', {'itemprop': 'numberOfPages'}).text.strip()
        return int(num_pages.split()[0])
    return ''


def get_year_first_published(soup):
    year_first_published = soup.find('nobr', attrs={'class':'greyText'})
    if year_first_published:
        year_first_published = year_first_published.string
        return re.search('([0-9]{3,4})', year_first_published).group(1)
    else:
        return ''

def get_id(bookid):
    pattern = re.compile("([^.-]+)")
    return pattern.search(bookid).group()
    
def scrape_book(book_id):
    url = 'https://www.goodreads.com/book/show/' + book_id
    source = urlopen(url)
    soup = bs4.BeautifulSoup(source, 'html.parser')

In [ ]:
print(len(title))
print(len(author))

3002
3002


In [ ]:
a = {'Book Name':title, 'Author Name':author,  }
book_data = pd.DataFrame(data=a)
book_data

,Book Name,Author Name
0,The Hitchhiker's Guide to the Galaxy (Hitchhik...,Douglas Adams
1,Little Women (Paperback),Louisa May Alcott
2,Pride and Prejudice (Paperback),Jane Austen
3,I Know Why the Caged Bird Sings (Maya Angelou'...,Maya Angelou
4,To Kill a Mockingbird (Paperback),Harper Lee
...,...,...
2997,A Tale of Two Cities (Paperback),Charles Dickens
2998,The Count of Monte Cristo (Paperback),Alexandre Dumas
2999,The Scarlet Letter (Paperback),Nathaniel Hawthorne
3000,Beloved (Paperback),Toni Morrison
